In [ ]:
!pip3 install openai

In [144]:
#Importing required libraries

from typing_extensions import override
from openai import AssistantEventHandler
from openai import OpenAI
import openai
import os 


In [145]:
#EventHandler class to handle events in the response stream and submit all tool outputs at once

class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    pass
    # print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)

      
  def on_tool_call_created(self, tool_call):
      print(f"\nassistant > {tool_call.type}\n", flush=True)

  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\code_intrepreter >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)


 


In [ ]:
open_ai_key = os.getenv('OPENAI_API_KEY')

In [86]:

# Initialize the OpenAI client
client = openai.Client(api_key=open_ai_key)

# Define the prompt
prompt = """
Based on the attached CSV file, list Tina, Eric, Paul, and Sarah's favorite city and explain in detail why they like it.
"""

# List of models to run the prompt on
models = ["gpt-3.5-turbo", "gpt-3.5-turbo-16k", "gpt-4", "gpt-4-turbo", "gpt-4o-mini", "gpt-4o" ]

async def upload_file (file_path=""):
    file_path = '/Users/abhis/Downloads/tse_takehome_dataset.csv'  # Replace with your actual file path
    file = client.files.create(
            file=open(file_path, "rb"),
            purpose='assistants'
            )
    print(file)
    file_id = file.id
    return file_id
file_id = await upload_file()
print(f"File uploaded successfully with file_id: {file_id}")


FileObject(id='file-G8Y5bgpsdS9esObaaSZqFTuR', bytes=12551, created_at=1726538358, filename='tse_takehome_dataset.csv', object='file', purpose='assistants', status='processed', status_details=None)
File uploaded successfully with file_id: file-G8Y5bgpsdS9esObaaSZqFTuR


In [124]:

# Step 1: Upload the CSV file and retrieve the file_id
try:
    file_id = await upload_file()
    print(f"File uploaded successfully with file_id: {file_id}")
except Exception as e:
    print(f"File upload failed: {e}")
    file_id = None  # In case of failure, file_id will be None

# Step 2: Ensure the file_id is available before proceeding
if not file_id:
    raise Exception("File ID is not defined. Ensure the file is uploaded successfully.")

File uploaded successfully with file_id: file-PmRcWhDm3Wnwm4ZYSfdWm7j6


In [119]:
#Create map assitants 

asst_arr = []

for model in models:
    assistant = client.beta.assistants.create(
            name=f"assistant_for_{model}",
            instructions="Testing multiple models for output comparison.",
            tools=[{"type": "code_interpreter"}],
            model=model,
            tool_resources={
                "code_interpreter": {
                    "file_ids": [file_id]  # Use the correct file_id here
                }
            }
        )

    asst_arr.append((model,assistant))

In [147]:
# gloabal thread for testing context window oveflow 
thread = client.beta.threads.create()

In [148]:
# Step 3: Create a function to generate responses from different models

def create_assistant_and_run_prompt(model, prompt, file_id):
    
    assistant = None
    # check for the model assitant 
    for asst_mdl,asst in asst_arr:
        if model== asst_mdl:
            assistant= asst

    # Create a thread and run the prompt
    thread = client.beta.threads.create()

    message = client.beta.threads.messages.create(
                thread_id=thread.id,
                role="user",
                content=prompt
                )
    with client.beta.threads.runs.stream(
          thread_id=thread.id,
          assistant_id=assistant.id,
          event_handler=EventHandler(),
        ) as stream:
            stream.until_done()



In [150]:
#Prompt function to create an Assistant and run

prompts = [ """
Based on the file data, tell me what is Eric rogers favorite city and explain in detail why he/she likes it.
""" ,  """
Based on the file data, tell me what is Ali' favorite city and explain in detail why he/she likes it.
""",  """
Based on the file data, tell me what is kristy alvarez's, Dr.joel simmon's favorite city and explain in detail why he/she likes it.
""", """
Based on the file data, tell me what is jessie crawford, douglas acosta, Dr.joel simmon's favorite city and explain in detail why he/she likes it.
""", """
Based on the file data, tell me what is lewis, edwards and king's favorite city and explain in detail why he/she likes it.
"""]
# loop through models and prompts 
for prompt in prompts:
    print("\n **********************************************************************************")
    print(f"\n THE input prompt is {prompt}")
    for model in models:
        print("\n ------------------------------------------------------------------------------")
        print(f"\n THIS is the model {model}" )

        create_assistant_and_run_prompt(model, prompt, file_id)
        print("\n ------------------------------------------------------------------------------")
    print("\n**********************************************************************************")



 **********************************************************************************

 THE input prompt is 
Based on the file data, tell me what is Eric rogers favorite city and explain in detail why he/she likes it.


 ------------------------------------------------------------------------------

 THIS is the model gpt-3.5-turbo
Eric Rogers' favorite city, according to the data in the uploaded file, is Paris. Eric Rogers likes Paris for its beautiful architecture and vibrant atmosphere. Paris is known for its stunning architectural landmarks, such as the Eiffel Tower, Notre-Dame Cathedral, and Louvre Museum, which contribute to the city's unique charm and historical significance.

Furthermore, Paris offers a vibrant atmosphere with its bustling streets, cultural richness, and diverse range of experiences. Eric Rogers likely appreciates the artistic culture, lively social scene, and the opportunity to explore art galleries, museums, and cafes while experiencing the city's unique ambia